In [241]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/my-multi-class-prediction-of-obesity-risk/data.csv
/kaggle/input/playground-series-s4e2/sample_submission.csv
/kaggle/input/playground-series-s4e2/train.csv
/kaggle/input/playground-series-s4e2/test.csv


# **肥満リスクの多クラス予測**
(Multi-Class Prediction of Obesity Risk)

# 参考資料
**書籍**


**Kaggle**
- The fat CatBoost😼

**自分で作成したファイル**
- 銀行解約データセットを使用した二項分類
- 肝硬変の転帰の多クラス予測

**その他**


In [242]:
import numpy as np
import pandas as pd

FILE_PATH = '/kaggle/input/playground-series-s4e2/'
CREATE_DATA = '/kaggle/input/my-multi-class-prediction-of-obesity-risk'


# 確認 (data_import.py)
def file_to_xy(filename):
    data = pd.read_csv(filename, index_col=0)
    print(f'読み込み完了 {filename}')
    
    target_col = 'NObeyesdad'
    
    train = data[:20758].reset_index(drop=True)
    test = data[20758:].reset_index(drop=True).drop(target_col, axis=1) # 目的変数を指定する
    # 目的変数と説明変数に分割
    X = train.drop([target_col], axis=1) # 目的変数を指定する
    y = train[target_col].values # 目的変数を指定する

    return data,test,train,X,y

filename = 'data.csv'
data,test,train,X,y = file_to_xy(CREATE_DATA + '/' + filename)

読み込み完了 /kaggle/input/my-multi-class-prediction-of-obesity-risk/data.csv


In [243]:
X = X.copy()
y = y.copy()

# 機械学習モデルの構築・学習・予測

## ライブラリのインポート

In [244]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 評価指標
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, recall_score, precision_score, confusion_matrix, mean_absolute_error, mean_squared_error, r2_score

# 機械学習モデル
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier


# 実験
import datetime
import pickle


sample_sub = pd.read_csv(FILE_PATH + "sample_submission.csv")

## データの分割

In [245]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

## ハイパーパラメータ

In [246]:
xgb_params = {'max_depth': 10,
    'min_child_weight': 1,
    'learning_rate': 0.733656719765889,
    'n_estimators': 696,
    'subsample': 0.7928524918864644,
    'colsample_bytree': 0.1345360036201174,
    'random_state': 42,
    'njobs': -1}

cat_params = {'iterations': 526,
    'depth': 3,
    'min_data_in_leaf': 13,
    'learning_rate': 0.0008029819794568197}

## モデルファイルと評価結果を出力する（ファイルの出力あり）, Main

In [247]:
def make_model_and_eval(model, X_train, X_test, y_train, y_test):
    model.fit(X_train, y_train)
    y_pred_train = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    # accuracy
    acc_train = accuracy_score(y_train, y_pred_train)
    acc_test = accuracy_score(y_test, y_pred_test)
    # F1_score
    f1_train = f1_score(y_train, y_pred_train, average='macro')
    f1_test = f1_score(y_test, y_pred_test, average='macro')
    # recall
    recall_train = recall_score(y_train, y_pred_train, average='macro')
    recall_test = recall_score(y_test, y_pred_test, average='macro')
    # precision
    precision_train = precision_score(y_train, y_pred_train, average='macro')
    precision_test = precision_score(y_test, y_pred_test, average='macro')
#    # AUC
#     auc_train = roc_auc_score(y_train, y_pred_train)
#     auc_test = roc_auc_score(y_test, y_pred_test)
   # MAE
    mae_train = mean_absolute_error(y_train, y_pred_train)
    mae_test = mean_absolute_error(y_test, y_pred_test)
   # MSE
    mse_train = mean_squared_error(y_train, y_pred_train)
    mse_test = mean_squared_error(y_test, y_pred_test)
   # RMSE
    rmse_train = np.sqrt(mse_train)
    rmse_test = np.sqrt(mse_test)
   # r2_score
    r2_train = r2_score(y_train, y_pred_train)
    r2_test = r2_score(y_test, y_pred_test)

#     tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_train).ravel()
#     tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
    score_train = pd.DataFrame({'DataCategory':['Train'],
#                                'auc':[auc_train],
                               'accuracy':[acc_train],
                               'f1':[f1_train],
                               'recall':[recall_train],
                               'precision':[precision_train],
                               'MAE':[mae_train],
                               'MSE':[mse_train],
                               'RMSE':[rmse_train],
                               'r2':[r2_train],
#                                'tp':[tp_train],
#                                'fn':[fn_train],
#                                'fp':[fp_train],
#                                'tn':[tn_train]
                               })
    score_test = pd.DataFrame({'DataCategory':['Valid'],
#                               'auc':[auc_test],
                              'accuracy':[acc_test],
                              'f1':[f1_test],
                              'recall':[recall_test],
                              'precision':[precision_test],
                              'MAE':[mae_test],
                              'MSE':[mse_test],
                              'RMSE':[rmse_train],
                              'r2':[r2_train],
#                               'tp':[tp_test],
#                               'fn':[fn_test],
#                               'fp':[fp_test],
#                               'tn':[tn_test]
                              })
    score = pd.concat([score_train, score_test], ignore_index=True)
    
    importance = pd.DataFrame({'cols': X_train.columns,
                              'importance': model.feature_importances_})
    importance = importance.sort_values('importance', ascending=False)
    cols = pd.DataFrame({'X_cols': X_train.columns})
    display(score)
    
    return score, importance, model, cols

In [248]:
now = datetime.datetime.now().strftime("%Y%m%d%H")
target_output_dir = 'results_' + now

os.makedirs(target_output_dir, exist_ok=True)
print(target_output_dir)



models = {'DecisionTreeClassifier': DecisionTreeClassifier(random_state=42), 
          'RandomForest':RandomForestClassifier(random_state=42),
          'GradientBoostingClassifier':GradientBoostingClassifier(random_state=42),
          'lightgbm': LGBMClassifier(random_state=42),
          'xgboost': XGBClassifier(random_state=42),
#           'BaggingClassifier': BaggingClassifier(random_state=42),
          'AdaBoostClassifier': AdaBoostClassifier(random_state=42),
#           'CatBoostClassifier': CatBoostClassifier(**cat_params),
         }

score_all = []
importance_all = []
for model_name, model in models.items():
    print(model_name)
    score, importance, model, cols = make_model_and_eval(model, X_train, X_valid, y_train, y_valid)
    score['model_name'] = model_name
    importance['model_name'] = model_name
    
    model_names = f'model_{model_name}.pickle'
    model_path = os.path.join(target_output_dir, model_names)
    with open(model_path, mode='wb') as f:
        pickle.dump(model, f, protocol=2)
    score_all.append(score)
    importance_all.append(importance)
score_all = pd.concat(score_all, ignore_index=True)
importance_all = pd.concat(importance_all, ignore_index=True)
cols = pd.DataFrame({'X_cols':X_train.columns})

score_name = 'score.csv'
importance_name = 'importance.csv'
cols_name = 'X_cols.csv'

score_path = os.path.join(target_output_dir, score_name)
importance_path = os.path.join(target_output_dir, importance_name)
cols_path = os.path.join(target_output_dir, cols_name)

score_all.to_csv(score_path, index=False)
importance_all.to_csv(importance_path, index=False)
cols.to_csv(cols_path, index=False)

results_2024020106
DecisionTreeClassifier


,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,1.0
1,Valid,0.847303,0.831943,0.831859,0.832185,0.376204,1.277457,0.0,1.0


RandomForest


,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.0,1.0
1,Valid,0.893064,0.881411,0.880445,0.882909,0.271195,0.942678,0.0,1.0


GradientBoostingClassifier


...

,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,0.979766,0.977217,0.977060,0.977446,0.051728,0.177827,0.421696,0.950175
1,Valid,0.903420,0.893093,0.892806,0.893439,0.239403,0.811175,0.421696,0.950175


xgboost


,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,0.985848,0.984105,0.984044,0.984209,0.035529,0.121884,0.349118,0.96585
1,Valid,0.904624,0.894678,0.894613,0.894786,0.240848,0.835260,0.349118,0.96585


AdaBoostClassifier


,DataCategory,accuracy,f1,recall,precision,MAE,MSE,RMSE,r2
0,Train,0.629411,0.567442,0.599230,0.621253,0.884500,2.879682,1.696963,0.193144
1,Valid,0.625482,0.558456,0.590388,0.609543,0.905588,2.980250,1.696963,0.193144


#### pickleファイルのインポート

In [249]:
import pickle

model_name = 'model_xgboost.pickle'

model_path = os.path.join(target_output_dir, model_name)

with open(model_path, mode='rb') as f:
    model = pickle.load(f)
    
print(model)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)


## モデルの予測

In [250]:
target_col = 'NObeyesdad'
sample_sub[target_col] = model.predict(test)

# 提出

In [251]:
inverse_mapping = {
                   0: 'Insufficient_Weight',
                   1: 'Normal_Weight', 
                   2: 'Obesity_Type_I',
                   3: 'Obesity_Type_II', 
                   4: 'Obesity_Type_III', 
                   5: 'Overweight_Level_I', 
                   6: 'Overweight_Level_II'
                  }

sample_sub[target_col] = sample_sub[target_col].map(inverse_mapping)
sample_sub.to_csv('submission.csv', index=False)
sample_sub.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


# 導入検討

### 標準化を行なったモデル

In [252]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

clf1 = LogisticRegression(penalty='l2',
                         C=0.001,
                         solver='lbfgs',
                         random_state=1)
clf2 = DecisionTreeClassifier(max_depth=1,
                             criterion='entropy',
                             random_state=0)
clf3 = KNeighborsClassifier(n_neighbors=1,
                           p=2,
                           metric='minkowski')

pipe1 = Pipeline([['sc', StandardScaler()],
                 ['clf', clf1]])
pipe3 = Pipeline([['sc', StandardScaler()],
                 ['clf', clf3]])

clf_labels = ['Logistic regression', 'Decision tree', 'KNN']

print('10-fold cross validation:\n')

for clf, label in zip([pipe1, clf2, pipe3], clf_labels):
    scores = cross_val_score(estimator=clf,
                            X=X_train,
                            y=y_train,
                            cv=10,
                            scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

10-fold cross validation:

Accuracy: 0.64 (+/- 0.01) [Logistic regression]
Accuracy: 0.34 (+/- 0.00) [Decision tree]
Accuracy: 0.73 (+/- 0.01) [KNN]
